In [47]:
'''A/B Testing for ShoeFly.com
Our favorite online shoe store, ShoeFly.com is performing an A/B Test. They have two different versions of an ad, 
which they have placed in emails, as well as in banner ads on Facebook, Twitter, and Google. 
They want to know how the two ads are performing on each of the different platforms on each day of the week. 
Help them analyze the data using aggregate measures.'''

import pandas as pd
import numpy as np


ad_clicks = pd.read_csv("ShoeFlycom_ab_testing.csv")
print(ad_clicks.head(10), "\n")

#How many views (i.e., rows of the table) came from each utm_source?
count_source_visits = ad_clicks.groupby("utm_source").user_id.count().reset_index()
print(count_source_visits, "\n")

#If the column ad_click_timestamp is not null, then someone actually clicked on the ad that was displayed.
#Create a new column called is_click, which is True if ad_click_timestamp is not null and False otherwise.
ad_clicks["is_click"] = ~ad_clicks.ad_click_timestamp.isnull()
print(ad_clicks.head(10), "\n")

#We want to know the percent of people who clicked on ads from each utm_source.
clicks_by_source = ad_clicks.groupby(["utm_source","is_click"]).user_id.count().reset_index()
print(clicks_by_source, "\n")

#Now let’s pivot the data so that the columns are is_click (either True or False), 
#the index is utm_source, and the values are user_id.
clicks_pivot = clicks_by_source.pivot(
    columns = "is_click",
    index = "utm_source",
    values = "user_id"
)#.reset_index()
print(clicks_pivot, "\n")

#Create a new column in clicks_pivot called percent_clicked 
#which is equal to the percent of users who clicked on the ad from each utm_source.

pd.options.display.float_format = "{:,.2f}".format
clicks_pivot["percent_clicked"] = clicks_pivot[True]/(clicks_pivot[True]+clicks_pivot[False])
print(clicks_pivot,"\n")

#The column experimental_group tells us whether the user was shown Ad A or Ad B.
#Were approximately the same number of people shown both adds?

which_ad = ad_clicks.groupby("experimental_group").user_id.count().reset_index()
print(which_ad, "\n")

#Using the column is_click that we defined earlier, check to see if a greater percentage of users clicked on Ad A or Ad B.
which_campaign = ad_clicks.groupby(["experimental_group", "is_click"]).user_id.count().reset_index()
which_campaign_pivot = which_campaign.pivot(
    columns="is_click",
    index="experimental_group",
    values="user_id"
)
print(which_campaign_pivot, "\n")

'''The Product Manager for the A/B test thinks that the clicks might have changed by day of the week.
Start by creating two DataFrames: a_clicks and b_clicks, 
which contain only the results for A group and B group, respectively.'''

a_clicks = ad_clicks[ad_clicks.experimental_group == "A"]
b_clicks = ad_clicks[ad_clicks.experimental_group == "B"]

#For each group (a_clicks and b_clicks), calculate the percent of users who clicked on the ad by day.
a_clicks_by_day = a_clicks.groupby(["is_click", "day"]).user_id.count().reset_index()
b_clicks_by_day = b_clicks.groupby(["is_click", "day"]).user_id.count().reset_index()

a_clicks_by_day_pivot = a_clicks_by_day.pivot(
    columns = "is_click",
    index = "day",
    values = "user_id"
)#.reset_index()
a_clicks_by_day_pivot["percent_clicked"] = a_clicks_by_day_pivot[True]/(a_clicks_by_day_pivot[True]+
                                                                        a_clicks_by_day_pivot[False])

b_clicks_by_day_pivot = b_clicks_by_day.pivot(
    columns = "is_click",
    index = "day",
    values = "user_id"
)
b_clicks_by_day_pivot["percent_clicked"] = b_clicks_by_day_pivot[True]/(b_clicks_by_day_pivot[True]+
                                                                        b_clicks_by_day_pivot[False])
print(a_clicks_by_day_pivot, "\n")
print(b_clicks_by_day_pivot, "\n")


                                user_id utm_source            day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google   6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook     7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter    2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google    2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook     7 - Sunday   
5  013b0072-7b72-40e7-b698-98b4d0c9967f   facebook     1 - Monday   
6  0153d85b-7660-4c39-92eb-1e1acd023280     google   4 - Thursday   
7  01555297-d6e6-49ae-aeba-1b196fdbb09f     google  3 - Wednesday   
8  018cea61-19ea-4119-895b-1a4309ccb148      email     1 - Monday   
9  01a210c3-fde0-4e6f-8efd-4f0e38730ae6      email    2 - Tuesday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  
3                NaN                  B  
4                NaN          